# scenario class init block

In [1]:
import json
import os
import pathlib
import shutil
import random

import pandas as pd
import numpy as np

from datetime import datetime
from pyproj import CRS, Transformer


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)


class Scenario():
    def __init__(self, beam_output_path, beam_crs, output_path):
        self.beam_crs = beam_crs
        self.beam_output_path = beam_output_path
        self.scenario_title = output_path.replace('_', ' ')
        
        # self.events_file = "events.csv"
        # self.events_layer_file = "events_settings.json"
        # self.network_layer_file = "network_settings.json"
        # self.trajectories_file = "trajectories.csv"
        # self.trajectories_layer_file = "trajectories_settings.json"
        # self.dynamic_network_layer_file = "dynamic_network_settings.json"
        
        self.layer_id = 0
        
        self.folder_name_events_icons = "EventIcons"
        self.folder_name_trajectory_icons = "TrajectoryIcons"
        self.image_folders_to_copy = ['image_folders/EventIcons', 'image_folders/TrajectoryIcons']
        
        self.layers = []
        
        self.events = []
        self.network = []
        self.trajectories = [] 
        self.dynamic_network = []
                
        self.trajectories_icons = [
            {"Type":"Car", "BackgroundColor":"FFA100", "Label":"Taxis", "Icon":"Diamond" },
            {"Type":"Pedastrian", "BackgroundColor":"FF0021", "Label":"Pedastrian", "Icon":"Triangle"},
            {"Type":"Bus", "BackgroundColor":"7C00FF", "Label":"Public Transportation", "Icon":"Circle"}
        ]
        self.events_icons = [
            {"Icon":"Flashing","Label":"Emergency"},
            {"Icon":"Solid", "Label":"Traffic Jam"}
        ]

        self.out_path = self.prepare_output_folder(output_path)

    
    def log(self, text):
        print(f" -> {text}")
        
        
    def copy_folder(self, src_folder, dest_folder):
        try:
            shutil.copytree(src_folder, dest_folder)
            self.log(f"'{src_folder}' copied to '{dest_folder}' successfully.")
        except shutil.Error as e:
            self.log(f"Error: {e}")
    
    
    def prepare_output_folder(self, output_folder_path):
        out_path = pathlib.Path(output_folder_path).resolve()

        try:
            if os.path.exists(out_path):
                shutil.rmtree(out_path)
                self.log(f"folder '{out_path}' and all its contents have been deleted.")
        except Exception as e:
            self.log(f"!! Failed to delete {file_path}. Reason: {e}")

        out_path.mkdir(exist_ok=True)
        
        for image_folder in self.image_folders_to_copy:
            image_folder_name = pathlib.Path(image_folder).resolve().name
            self.copy_folder(image_folder, out_path / image_folder_name)
            
        self.log(f"the output path set to '{out_path}'")
        return out_path

    
    def read_beam_output(self):
        self.log("reading beam output folder")
        self.read_network()

    
    def read_network(self):
        in_network_path = self.beam_output_path + "/network.csv.gz"
        in_network = pd.read_csv(in_network_path)

        crs_to = CRS.from_epsg(4326) # the lat lon CRS
        crs_from = CRS.from_epsg(self.beam_crs) # original map crs
        transformer = Transformer.from_crs(crs_from, crs_to)

        def xy_to_latlon(df_row):
            (from_x, from_y) = transformer.transform(df_row['fromLocationX'], df_row['fromLocationY'])
            (to_x, to_y) = transformer.transform(df_row['toLocationX'], df_row['toLocationY'])
            return df_row["linkId"], from_x, from_y, to_x, to_y

        network = pd.DataFrame()
        network_cols = ["linkId", "fromLocationX", "fromLocationY", "toLocationX", "toLocationY"]
        network[network_cols] = in_network.apply(xy_to_latlon, axis=1, result_type="expand")
        network["linkId"] = pd.to_numeric(network["linkId"], downcast='integer')
        
        self.log(f"read network ({len(network.index)}) from '{in_network_path}'")
        self.add_network(network)

    
    def add_network(self, network):
        self.log(f"adding network with {len(network)} records")
        
        network_file = f"network_{self.layer_id}.csv"
        network_config_file = f"network_{self.layer_id}_settings.json"
        layer = {
            "LayerName": f"Network_{self.layer_id}",
            "LayerType": 0,
            "OrderId": self.layer_id,
            "FileName": network_config_file,
            "Visible": True
        }
        
        self.layer_id += 1
        self.layers.append(layer)
        self.network.append((network, network_file, network_config_file))

        
    def add_events(self, events, events_icons, suffix=""):
        self.log(f"adding events with {len(events)} records, with {len(events_icons)} icons")
        
        csv_file = f"events_{suffix}_{self.layer_id}.csv"
        config_file = f"events_{suffix}_{self.layer_id}_settings.json"
        layer = {
            "LayerName": f"Event_{suffix}_{self.layer_id}",
            "LayerType": 2,
            "OrderId": self.layer_id,
            "FileName": config_file,
            "Visible": True
        }
        
        self.layer_id += 1
        self.layers.append(layer)
        self.events.append((events, events_icons, csv_file, config_file))
        

    def add_trajectory(self, trajectory, trajectory_icons, suffix=""):
        self.log(f"adding trajectory with {len(trajectory)} records, with {len(trajectory_icons)} icons")
        
        csv_file = f"trajectory_{suffix}_{self.layer_id}.csv"
        config_file = f"trajectory_{suffix}_{self.layer_id}_settings.json"
        layer = {
            "LayerName": f"Trajectory_{suffix}_{self.layer_id}",
            "LayerType": 1,
            "OrderId": self.layer_id,
            "FileName": config_file,
            "Visible": True
        }
        
        self.layer_id += 1
        self.layers.append(layer)
        self.trajectories.append((trajectory, trajectory_icons, csv_file, config_file))

        
    def add_dynamic_network(self, network):
        self.log(f"adding dynamic network with {len(network)} records")
        
        network_file = f"dynamic_network_{self.layer_id}.csv"
        network_config_file = f"dynamic_network_{self.layer_id}_settings.json"
        layer = {
            "LayerName": f"DynamicNetwork_{self.layer_id}",
            "LayerType": 3,
            "OrderId": self.layer_id,
            "FileName": network_config_file,
            "Visible": True
        }
        self.layer_id += 1
        self.layers.append(layer)
        self.dynamic_network.append((network, network_file, network_config_file))
        
        
    def add_empty_events(self):
        events = pd.DataFrame(columns=['LinkId', 'StartTime', 'EndTime', 'Type'])
        self.add_events(events, [], "empty")
        

    def add_empty_trajectories(self):
        trajectories = pd.DataFrame(columns=['ObjectId', 'Type', 'ProgressBarType', 'ExitTimeLastLink', 'Path'])
        self.add_trajectory(trajectories, [], "empty")


    def add_empty_dynamic_network(self):
        dynamic_network = pd.DataFrame(columns=['LinkId', 'EndTime', 'AnimationSequence'])
        self.add_dynamic_network(dynamic_network)
        
        
    def write_config(self): 
        config = {
            "WindowTitle": self.scenario_title,
            "SimulationTimeSpeed": 5.0,
            "EndSimulationTime": 1600,
            "MapBoxAPIAccessToken": "pk.eyJ1IjoieXVuZWViOTAiLCJhIjoiY2x2ZHE3NjR4MDFvNjJubzBta2ZmaHo3aCJ9.5leK6jjiUYupP1b8DeiMLw",
            "Layers": self.layers
        }
        config_path = str((self.out_path / "config.json").resolve())
        with open(config_path, "w") as file:
            file.write(json.dumps(config))
    
    
    def write_network_with_settings(self):
        while self.network:
            network, network_file_name, network_layer_file = self.network.pop()
            
            network_path = (self.out_path / network_file_name).resolve()
            network.to_csv(network_path, index=False)
            self.log(f"network written to '{network_path}'")

            network_settings = { "NetworkWidth":6, "NetworkColor":"55DDAA", "SelectionColor":"FF8A00", "FileName":network_file_name }
            network_settings_path = str((self.out_path / network_layer_file).resolve())
            with open(network_settings_path, "w") as file:
                file.write(json.dumps(network_settings))
                
            self.log(f"network settings written to {network_settings_path}")    

        
    def write_dynamic_network_with_settings(self):
        while self.dynamic_network:
            dynamic_network, network_file_name, dynamic_network_layer_file = self.dynamic_network.pop()

            network_path = (self.out_path / network_file_name).resolve()
            dynamic_network.to_csv(network_path, index=False)
            self.log(f"dynamic network written to {network_path}")

            network_settings = { "FileName": network_file_name }
            network_settings_path = str((self.out_path / dynamic_network_layer_file).resolve())
            with open(network_settings_path, "w") as file:
                file.write(json.dumps(network_settings))
            
            self.log(f"dynamic network settings written to {network_settings_path}")

        
    def write_trajectories_with_settings(self):
        while self.trajectories:
            trajectories, trajectories_icons, trajectories_file, trajectories_layer_file = self.trajectories.pop()

            path_to_output_file = str((self.out_path / trajectories_file).resolve())
            trajectories.to_csv(path_to_output_file, index=False)
            self.log(f"{len(trajectories.index)} trajectories written to {path_to_output_file} ...")

            trajectories_settings = {
                "IconAlignmentType": "Perpendicular",
                "IconZoomScaleFactor": 800,
                "IconFolderPath": self.folder_name_trajectory_icons,
                "IconConfig": trajectories_icons,
                "FileName": trajectories_file   
            }

            trajectories_settings_path = str((self.out_path / trajectories_layer_file).resolve())
            with open(trajectories_settings_path, "w") as file:
                file.write(json.dumps(trajectories_settings))

            self.log(f"trajectories settings written to {trajectories_settings_path}")

        
    def write_events_with_settings(self):
        while self.events:
            events, events_icons, events_file, events_layer_file = self.events.pop()

            path_to_output_file = str((self.out_path / events_file).resolve())
            events.to_csv(path_to_output_file, index=False)
            self.log(f"{len(events.index)} events written to {path_to_output_file} ...")

            events_settings = {
                "IconZoomScaleFactor":1600,
                "IconFolderPath": self.folder_name_events_icons,
                "IconConfig": events_icons,
                "FileName": events_file
            }

            events_settings_path = str((self.out_path / events_layer_file).resolve())
            with open(events_settings_path, "w") as file:
                file.write(json.dumps(events_settings))

            self.log(f"events settings written to {events_settings_path}")

        
    def write_network(self):
        self.write_network_with_settings()
        self.write_dynamic_network_with_settings()
        
        
    def write_scenario(self):
        self.write_config()
        self.write_network_with_settings()
        self.write_dynamic_network_with_settings()
        self.write_trajectories_with_settings()
        self.write_events_with_settings()
        self.log(f"scenario files written to {self.out_path}")
        
        
    def add_trajectories_from_pte(self, PTE_df, icon_settings, suffix=""):

        def path_traversal_to_lastlinktime_path(path_traversal_event):
            links = path_traversal_event['links'].split(',')
            link_travel_time = path_traversal_event['linkTravelTime'].split(',')

            departure = path_traversal_event['departureTime']

            link_enter_time = link_travel_time[:-1]
            link_enter_time.insert(0, departure)

            # path is string f"{enterTime}_{linkId}"
            path = []
            float_travel_times = []
            total_travel_time = 0
            for (link, str_time) in zip(links, link_enter_time):
                travel_time = float(str_time)
                float_travel_times.append(travel_time)
                total_travel_time = round(travel_time + total_travel_time, 2)
                path.append(f"{total_travel_time}_{link}")

            last_or_minimum = max(float(link_travel_time[-1]), min(float_travel_times))
            exit_time_last_link = round(last_or_minimum + total_travel_time, 2)

            return exit_time_last_link, "+".join(path)        
        
        trajectories = pd.DataFrame(columns=['ObjectId', 'Type', 'ExitTimeLastLink', 'Path'])

        trajectories['Type'] = PTE_df['Type']
        trajectories['ObjectId'] = PTE_df['vehicle']
        trajectories[['ExitTimeLastLink', 'Path']] = PTE_df.apply(path_traversal_to_lastlinktime_path, axis=1, result_type="expand")
        
        self.log(f"got {len(trajectories.index)} trajectories")
        self.add_trajectory(trajectories, icon_settings, suffix)

        
    def pack_to_archive(self, archive_type='zip'):
        source = str(self.out_path)
        destination = f"{source}.{archive_type}"
   
        base, name = os.path.split(destination)
        archive_from = os.path.dirname(source)
        archive_to = os.path.basename(source.strip(os.sep))
        
        shutil.make_archive(name, archive_type, archive_from, archive_to)
        shutil.move('%s.%s' % (name, archive_type), destination)
        shutil.rmtree(source)
        
        self.log(f"scenario packed to '{destination}'")

        
# ## create an empty scenario with map and without events\trajectories
# beam_output = "../beam_root/output/sf-light/sf-light-1k-xml__2024-05-06_18-09-08_xjf"
# output_folder_path = "out_" + beam_output.split('/')[-1].split("\\")[-1]
# beam_crs = 26910
# scenario = Scenario(beam_output, beam_crs, output_folder_path)
# scenario.read_beam_output()
# scenario.write_scenario()
# scenario.pack_to_archive()


# ## pack output folder to tar.gz
# out = scenario.out_path.name
# archive_name = f"{out}_rh_passengers.tar.gz"
# ! rm -rf "$archive_name"
# ! rm -rf "$out/.ipynb"*
# ! tar -zcvf "$archive_name" "$out"
# ! ls "$archive_name" -lh


print("initialized")

initialized


# independant functions init block

In [2]:
### a set of functions to read and process BEAM events

def read_events(path_to_events_file, event_types=None, nrows=None):
    events_dtype = { 
        'riders' : str,
        'driver' : str,
        'vehicle' : str,
        'person' : str,
        'links': str
    }

    event_types_to_read = set()
    filter_by_type = False
    
    if event_types:
        event_types_to_read = set(event_types)
        filter_by_type = True
    
    df_list = []
    chunksize = 10 ** 6
    with pd.read_csv(path_to_events_file, dtype=events_dtype, low_memory=False, chunksize=chunksize, nrows=nrows) as reader:
        for chunk in reader:
            if filter_by_type:
                df = chunk[chunk['type'].isin(event_types_to_read)]
            else:
                df = chunk
                
            df_list.append(df)
    
    events1 = pd.concat(df_list).dropna(axis=1, how='all')
    return events1


def get_events_file_path(beam_output, iteration):
    p1 = (pathlib.Path(beam_output) / f"ITERS/it.{iteration}/{iteration}.events.csv").resolve()
    p2 = (pathlib.Path(beam_output) / f"ITERS/it.{iteration}/{iteration}.events.csv.gz").resolve()
    if p1.is_file():
        return str(p1)
    elif p2.is_file():
        return str(p2)
    else:
        raise Exception(f"Events file does not exist! Not '{str(p1)}' nor '{str(p2)}'")

        
def read_pte_events(beam_output, iteration, nrows=None):
    path_to_events_file = get_events_file_path(beam_output, iteration)                               
    # print(f"reading events from {path_to_events_file} ...")
    ptes = read_events(path_to_events_file, event_types = ["PathTraversal"], nrows=nrows)
    with_links = ptes['links'].notna()
    all_pte = ptes[with_links].copy()
    # print(f"read {len(all_pte)} PathTraversal events")
    return all_pte


def get_trips(events_df):

    def get_mode_or_action(row):
        event_type = row['type']
        if event_type == 'actend' or event_type == 'actstart':
            return "A:"
        elif event_type == 'PersonEntersVehicle':
            return "V:"  + row['vehicle']
        
        print(f"Unexpected event type: {event_type}")
        return np.nan

    
    def get_sequences_of_vehicles_and_indexes_per_trip(row):
        action_sequence = row['sequence']
        index_sequence = row['index']
        
        result_action_seq = []
        result_index_seq = []
        
        vehicle_used = []
        index_used = []

        for (step, idx) in zip(action_sequence, index_sequence):
            if step.startswith("A:"):
                if len(vehicle_used) > 0:
                    result_action_seq.append(vehicle_used)
                    if index_used[-1] != idx:
                        index_used.append(idx)
                        
                    result_index_seq.append(index_used)

                vehicle_used = []
                index_used = [idx]
                
            if step.startswith("V:"):
                vehicle_used.append(step[2:])
                index_used.append(idx)

        if len(result_action_seq) > 0 and len(result_index_seq) > 0:
            return result_action_seq, result_index_seq
        else:
            return np.nan, np.nan
    

    ## get mode choice and person enters vehicle events
    selected_types = set(['actend', 'actstart', 'PersonEntersVehicle'])
    is_type = events_df['type'].isin(selected_types)
    events_df2 = events_df[is_type].dropna(axis=1, how='all')
    
    if len(events_df2) == 0:
        return pd.DataFrame()
    
    ## addind field 'sequence' with vehicle types and modes for selected events
    events_df2['sequence'] = events_df2.apply(get_mode_or_action, axis=1)
    
    ## group by person 
    persons_df = events_df2.groupby('person')[['index','sequence']].agg(list)
    
    ## transform sequence of modes and vehicles into lists of lists (vehicles, events indexes)
    persons_df[['vehicles_sequence', 'index_sequence']] = persons_df.apply(get_sequences_of_vehicles_and_indexes_per_trip, axis=1, result_type="expand")
    persons_df.dropna(subset=['vehicles_sequence','index_sequence'], how='all', inplace=True)
    
    ## explode DF in order to have one row per trip with pair: mode, used vehicles
    persons_vehicles_sequences = persons_df.explode(['vehicles_sequence', 'index_sequence'])[['vehicles_sequence', 'index_sequence']]
    
    one_trip_per_row_df = persons_vehicles_sequences \
        .reset_index() \
        .reset_index() \
        .rename(columns={'index': 'trip_id', 'index_sequence':'index', "vehicles_sequence": "trip_vehicles"})
    
    return one_trip_per_row_df


def add_vehicle_type_to_all_events_with_vehicles(events_df):
    original_columns = list(events_df.columns) + ['index']
    
    vehicle_not_na = events_df['vehicle'].notna()
    vehicle_type_not_na = events_df['vehicleType'].notna()
    vehicle_to_mode = events_df[vehicle_not_na & vehicle_type_not_na].groupby('vehicle')['vehicleType'].first()
    
    events_df_1 = events_df.drop(columns='vehicleType').reset_index()
    merged_events = events_df_1.merge(vehicle_to_mode, how='outer', on='vehicle')[original_columns].set_index('index')
    return merged_events


def sort_events(events_df):
    ## all events have default order
    events_df['order'] = 7
    
    body_vehicles = set(events_df[events_df['vehicleType'] == 'BODY-TYPE-DEFAULT']['vehicle'].unique())
    
    ## changing order of all events of specified type
    events_df.loc[events_df['type'] == 'actend', 'order'] = 2
    events_df.loc[events_df['type'] == 'PersonEntersVehicle', 'order'] = 3
    events_df.loc[events_df['type'] == 'PersonLeavesVehicle', 'order'] = 4
    events_df.loc[(events_df['type'] == 'PersonLeavesVehicle') & (events_df['vehicle'].isin(body_vehicles)), 'order'] = 8
    events_df.loc[events_df['type'] == 'actstart', 'order'] = 10
    
    ## fixing time of PathTraversal events
    events_df.loc[events_df['type'] == 'PathTraversal', 'time'] = events_df.loc[events_df['type'] == 'PathTraversal', 'departureTime']
    
    ## ordering events by time and then order 
    return events_df.sort_values(['time', 'order'])    


def add_person_to_path_traversal(events_df):
    is_pte_with_riders = (events_df['type'] == 'PathTraversal') & (events_df['riders'].notna())
    pte_df = events_df[is_pte_with_riders].copy()
    rest_df = events_df[~is_pte_with_riders].copy()
    
    pte_df['person'] = pte_df.apply(lambda r: r['riders'].split(":"), axis=1)
    pte_df_one_per_person = pte_df.explode('person')
    
    events_df_merged = pd.concat([pte_df_one_per_person, rest_df])
    
    return events_df_merged


def add_trip_id(events_df):
    if 'index' not in events_df.columns:
        events_df.reset_index(inplace=True)
        
    if 'trip_id' in events_df.columns:
        print("NOT CHANGING ANYTHING, 'trip_id' column already present in events DF!!")
        return events_df
    
    ## getting trips, a df with one trip per row [ trip_id person trip_vehicles index ]
    one_trip_per_row_df = get_trips(events_df)

    ## explode to have one row per event index
    one_row_per_event_df = one_trip_per_row_df.explode('index').drop(columns=['person'])

    ## merge original events into trip list
    events_with_trips = pd.merge(events_df, one_row_per_event_df, left_on='index', right_on='index', how='outer')
    events_with_trips.insert(2, 'trip_id', events_with_trips.pop('trip_id'))
        
    return (sort_events(events_with_trips), one_trip_per_row_df)


def fix_person_enters_leaves_rh_body_events(original_df_trip):
    df_trip = original_df_trip.copy()
    is_rh = df_trip['vehicle'].str.contains('rideHailVehicle')
    is_body = df_trip['vehicle'].str.contains('body')
    is_pte = df_trip['type'] == 'PathTraversal'
    unique_vehicles = df_trip[(is_rh | is_body) & is_pte]['vehicle'].unique()
    
    for vehicle_id in unique_vehicles:
        is_vehicle = df_trip['vehicle'] == vehicle_id
        rh_ptes = df_trip[is_vehicle & is_pte]
        
        min_departure_time = rh_ptes['departureTime'].min()
        max_arrival_time = rh_ptes['arrivalTime'].max()
        
        df_trip.loc[is_vehicle & (df_trip['type'] == 'PersonEntersVehicle'), 'time'] = min_departure_time
        df_trip.loc[is_vehicle & (df_trip['type'] == 'PersonLeavesVehicle'), 'time'] = max_arrival_time
        
    return df_trip


def fix_pte_walk_events(original_df_trip):
    df_trip = original_df_trip.copy()
    
    ptes = df_trip[df_trip['type'] == 'PathTraversal'].copy()
    ptes['is_body'] = ptes.apply(lambda r: 'body' in r['vehicle'], axis=1)
    ptes['is_rh'] = ptes.apply(lambda r: 'rideHailVehicle' in r['vehicle'], axis=1)
    ptes['is_rh_next'] = ptes['is_rh'].shift(-1)
    ptes['departure_next'] = ptes['departureTime'].shift(-1)
    
    is_walk = ptes['is_body'] == True
    rh_next = ptes['is_rh_next'] == True
    time_is_wrong = ptes['departure_next'] < ptes['arrivalTime']
    
    wrong_walk_pte = ptes[is_walk & rh_next & time_is_wrong].copy()
    wrong_walk_pte['time_shift'] = wrong_walk_pte.apply(lambda r: r['departure_next'] - r['arrivalTime'], axis=1)

    df_trip_2 = pd.merge(df_trip, wrong_walk_pte[['index', 'time_shift']], left_on='index', right_on='index', how='outer')
    
    def shift_time(row):
        time_shift = row['time_shift']
        if pd.notna(time_shift):
            for col in ['time', 'departureTime', 'arrivalTime']:
                row[col] += time_shift

        return row
    
    df_trip_2 = df_trip_2.apply(shift_time, axis=1)
    
    columns_to_remove = set(df_trip_2.columns) - set(original_df_trip.columns)
    df_trip_3 = df_trip_2.drop(columns = columns_to_remove)
    
    return df_trip_3
    
    
def fix_act_end(original_df_trip):
    df_trip = original_df_trip.copy()
    
    min_time = df_trip['time'].min()
    is_act_end = df_trip['type'] == 'actend'
    
    df_trip.loc[is_act_end, 'time'] = min_time
    
    return df_trip


def get_trip_df(events_df, selected_trip_id):
    if 'trip_id' not in events_df.columns:
        raise Exception(f"The input DF does not have 'trip_id' column!")
        
    # condition to select events with set trip ID
    trip_condition = events_df['trip_id'] == selected_trip_id
    
    trip_events = events_df[trip_condition]
    used_persons = list(trip_events['person'].unique())
    used_vehicles = set(trip_events['vehicle'].unique())
    
    allowed_time_delta = 100
    time_min = trip_events['time'].min() - allowed_time_delta
    time_max = trip_events['time'].max() + allowed_time_delta
    
    if len(used_persons) > 1:
        raise Exception("Too many persons in one trip: " + ", ".join(selected_persons_list))
    if len(used_persons) < 1:
        raise Exception("There are 0 persons in the selected trip")
    
    selected_person = used_persons[0]
    
    is_pte = events_df['type'] == 'PathTraversal'
    pte_within_time_window = (events_df['departureTime'] > time_min) & (events_df['departureTime'] < time_max)
    
    person_is_rider = events_df['person'] == selected_person
    person_is_driver = events_df['driver'] == selected_person
    person_driver_or_rider = person_is_rider | person_is_driver
    
    # condition to select PathTraversal events
    pte_condition = is_pte & pte_within_time_window & person_driver_or_rider

    is_plv = events_df['type'] == 'PersonLeavesVehicle'
    person_selected = events_df['person'] == selected_person
    within_time_window = (events_df['time'] > time_min) & (events_df['time'] < time_max)
    
    # condition to select PersonLeavesVehicle events
    plv_condition = is_plv & person_selected & within_time_window

    trip_data_frame = events_df[trip_condition | pte_condition | plv_condition]
    
    # fixes for wrong time in events when RH with stops feature enabled
    fixed_trip_df_1 = fix_pte_walk_events(trip_data_frame)
    fixed_trip_df_2 = fix_person_enters_leaves_rh_body_events(fixed_trip_df_1)
    fixed_trip_df_3 = fix_act_end(fixed_trip_df_2)

    return sort_events(fixed_trip_df_3)


#
# HOW TO USE read_events_enhanced_events_trips function
#
## 1. execute code in cell 1 with correct path to events 
## 2. execute code in cell 2
## 3. execute code in cell 3 (optionally change selected_trip_id in cell 3)
#
## results of cell 1 include: whole original events DF, enhanced events DF and all trips DF
## enhanced events has trip_id for some events types, improved order and one PathTraversal event per rider (instead of one PathTraversal event per vehicle)
## %%time - is a magic command which calculates how long the execution of the whole cell took, this command should be the first row in a cell
#
# %%time
# ## CELL 1 - read all events
# path1 = "../beam_root/output/sf-light/multiple_rhm__2023-11-09_19-15-47_aet/ITERS/it.0/0.events.csv.gz"
# events_original, events, all_trips = read_events_enhanced_events_trips(path1)
# print(f"Size of original events DF: {len(events_original)}, enhanced events DF: {len(events)}, all trips DF: {len(all_trips)}")
# display(events_original.head(2))
#
# %%time
# ### CELL 2 - find the required trip to use
# is_vehicle = all_trips['trip_vehicles'].str.contains('some-vehicle-id')
# selected_trips = all_trips[is_vehicle]
# all_trip_ids = list(all_trips['trip_id'].unique())
# selected_trip_ids = list(selected_trips['trip_id'].unique())
# display(f"Trips selected: {len(selected_trip_ids)}")
# display(selected_trips.head(2))
#
# %%time
# ### CELL 3 - show selected trip
# selected_trip_id = random.choice(selected_trips)
# trip_df = get_trip_df(events, selected_trip_id)
# display(f"Trip Id {selected_trip_id}, number of events in it: {len(trip_df)}")
# columns = ['person', 'trip_id', 'type', 'vehicle', 'mode', 'time', 'departureTime', 'arrivalTime', 'length']
# display(trip_df[columns])
#
def read_events_enhanced_events_trips(path_to_events_file):
    ## reading all events without changes
    all_events = read_events(path_to_events_file)
    
    ## adding vehicle type to all events with vehicle, for correct sorting
    ## all_events_with_vehicle_type = add_vehicle_type_to_all_events_with_vehicles(all_events)

    ## cloning PathTraversal events to have one PTE events per rider
    events_with_one_pte_per_rider = add_person_to_path_traversal(all_events)

    ## adding trip id to actend, actstart and PersonEntersVehicle events
    events, all_trips = add_trip_id(events_with_one_pte_per_rider)
    
    ## return all events, enhanced event and all trips
    return (all_events, events, all_trips)



print("initialized")

initialized


# scenarios

## an empty scenario with map and without events\trajectories

In [20]:
beam_output = "../beam_root/output/sf-light/sf-light-1k-xml__2024-08-20_14-17-57_vcp"

output_folder_path = "sflight-1k-network-only"
beam_crs = 26910

scenario = Scenario(beam_output, beam_crs, output_folder_path)
scenario.read_beam_output()

scenario.add_empty_events()
scenario.add_empty_trajectories()
scenario.add_empty_dynamic_network()

scenario.write_scenario()
scenario.pack_to_archive()

 -> folder '/home/jovyan/visualisation/sflight-1k-network-only' and all its contents have been deleted.
 -> 'image_folders/EventIcons' copied to '/home/jovyan/visualisation/sflight-1k-network-only/EventIcons' successfully.
 -> 'image_folders/TrajectoryIcons' copied to '/home/jovyan/visualisation/sflight-1k-network-only/TrajectoryIcons' successfully.
 -> the output path set to '/home/jovyan/visualisation/sflight-1k-network-only'
 -> reading beam output folder
 -> read network (94350) from '../beam_root/output/sf-light/sf-light-1k-xml__2024-08-20_14-17-57_vcp/network.csv.gz'
 -> adding network with 94350 records
 -> adding events with 0 records, with 0 icons
 -> adding trajectory with 0 records, with 0 icons
 -> adding dynamic network with 0 records
 -> network written to '/home/jovyan/visualisation/sflight-1k-network-only/network_0.csv'
 -> network settings written to /home/jovyan/visualisation/sflight-1k-network-only/network_settings_0.json
 -> dynamic network written to /home/jovyan/v

## all RH PT events split into 3 groups: without passengers, with 1 passenger, with more passengers

In [4]:
beam_output = "../beam_root/output/sf-light/sf-light-1k-xml__2024-05-06_18-09-08_xjf"
output_folder_path = "sflight-1k-rh_passengers"
beam_crs = 26910

scenario = Scenario(beam_output, beam_crs, output_folder_path)
scenario.read_beam_output()

scenario.add_empty_events()
scenario.add_empty_dynamic_network()

all_pte = read_pte_events(beam_output, 0)
is_rh = all_pte['vehicleType'] == "RH_Car"
all_rh = all_pte[is_rh]

rh_icons = [
    {
        "Type":"RH0",
        "BackgroundColor":"c4c4c4",
        "Label":"RH without passengers",
        "Icon":"Triangle"
    },
    {
        "Type":"RH1",
        "BackgroundColor":"fccf03",
        "Label":"RH with 1 passenger",
        "Icon":"Triangle"
    },
    {
        "Type":"RHM",
        "BackgroundColor":"fc0f03",
        "Label":"RH with more than 1 passenger",
        "Icon":"Triangle"
    }
]

def pte_to_icon_type(path_traversal_event):
    num_passengers = path_traversal_event['numPassengers']
    if num_passengers < 1.0:
        return "RH0"
    elif num_passengers == 1.0:
        return "RH1"
    else:
        return "RHM"

def pte_to_progress_bar(pte):
    return "None"

    
scenario.set_trajectoris(all_rh, pte_to_icon_type, pte_to_progress_bar, rh_icons)
scenario.write_scenario()
# scenario.pack_to_archive()

 -> the output path set to '/home/jovyan/visualisation/sflight-1k-rh_passengers'
 -> reading beam output folder
 -> read network (94350) from '../beam_root/output/sf-light/sf-light-1k-xml__2024-05-06_18-09-08_xjf/network.csv.gz'
 -> got 5767 trajectories
 -> network written to '/home/jovyan/visualisation/sflight-1k-rh_passengers/network.csv'
 -> network settings written to /home/jovyan/visualisation/sflight-1k-rh_passengers/network_settings.json
 -> dynamic network written to /home/jovyan/visualisation/sflight-1k-rh_passengers/dynamic_network.csv
 -> dynamic network settings written to /home/jovyan/visualisation/sflight-1k-rh_passengers/dynamic_network_settings.json
 -> 5767 trajectories written to /home/jovyan/visualisation/sflight-1k-rh_passengers/trajectories.csv ...
 -> trajectories settings written to /home/jovyan/visualisation/sflight-1k-rh_passengers/trajectories_settings.json
 -> 0 events written to /home/jovyan/visualisation/sflight-1k-rh_passengers/events.csv ...
 -> events s

## ALL events to three layers - RH, BUS, others

In [3]:
beam_output = "../beam_root/output/sf-light/sf-light-1k-xml__2024-08-20_14-17-57_vcp"
output_folder_path = "sflight-1k_rh_bus_other"
# beam_output = "../downloaded_data/sfbay/sfbay-freight-23Jan24-Base__2024-01-31_18-10-36_gfh"
# output_folder_path = "sfbay-freight-23Jan24-Base-rh_passengers"

beam_crs = 26910

scenario = Scenario(beam_output, beam_crs, output_folder_path)
scenario.read_beam_output()

scenario.add_empty_events()
scenario.add_empty_dynamic_network()

scenario.write_network()

 -> folder '/home/jovyan/visualisation/sflight-1k_rh_bus_other' and all its contents have been deleted.
 -> 'image_folders/EventIcons' copied to '/home/jovyan/visualisation/sflight-1k_rh_bus_other/EventIcons' successfully.
 -> 'image_folders/TrajectoryIcons' copied to '/home/jovyan/visualisation/sflight-1k_rh_bus_other/TrajectoryIcons' successfully.
 -> the output path set to '/home/jovyan/visualisation/sflight-1k_rh_bus_other'
 -> reading beam output folder
 -> read network (94350) from '../beam_root/output/sf-light/sf-light-1k-xml__2024-08-20_14-17-57_vcp/network.csv.gz'
 -> adding network with 94350 records
 -> adding events with 0 records, with 0 icons
 -> adding dynamic network with 0 records
 -> network written to '/home/jovyan/visualisation/sflight-1k_rh_bus_other/network_0.csv'
 -> network settings written to /home/jovyan/visualisation/sflight-1k_rh_bus_other/network_0_settings.json
 -> dynamic network written to /home/jovyan/visualisation/sflight-1k_rh_bus_other/dynamic_networ

In [4]:
all_pte = read_pte_events(beam_output, 0)
print(all_pte.shape)
display(all_pte['mode'].value_counts())
all_pte.head(2)

(364656, 29)


bus     354280
car      10035
walk       341
Name: mode, dtype: int64

,time,type,currentTourMode,vehicleType,links,numPassengers,length,primaryFuel,riders,fromStopIndex,seatingCapacity,tollPaid,secondaryFuelLevel,primaryFuelLevel,endY,endX,startY,startX,capacity,arrivalTime,departureTime,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,toStopIndex,driver,vehicle,mode
16036,14853.0,PathTraversal,NaN,BUS-DEFAULT,"51087,50223,51085,94245,51083,51081,47225,88927,47223",0.0,518.513,1.039515e+07,NaN,0.0,19.0,0.0,0.0,2.998960e+10,37.744058,-122.420890,37.748574,-122.418106,29.0,14853.0,14760.0,"5.875,5.875,5.875,5.875,5.875,5.875,5.875,5.875,0",0.0,None,Diesel,1.0,TransitDriverAgent-SF:7596499,SF:7596499,bus
16037,14923.0,PathTraversal,NaN,BUS-DEFAULT,"47223,47221,47219,47217,88759",0.0,205.071,4.111263e+06,NaN,1.0,19.0,0.0,0.0,2.998549e+10,37.742415,-122.421952,37.745036,-122.420258,29.0,14923.0,14853.0,"8.5,8.5,8.5,8.5,0",0.0,None,Diesel,2.0,TransitDriverAgent-SF:7596499,SF:7596499,bus


### RH PT into 3 groups:withpassengers, dead heading, repositioning

In [5]:
is_rh = all_pte['driver'].str.startswith('rideHailAgent')
all_rh = all_pte[is_rh].copy()

print(f" ->> total number of RH rows in DF {len(all_rh)}, all rows in DF {len(all_pte)}")

 ->> total number of RH rows in DF 8461, all rows in DF 364656


In [7]:
vehicle_to_passengers = {}
# going backwards
for idx, row in all_rh.sort_values('time', ascending=False).iterrows():
    v = row['vehicle']
    all_rh.loc[idx, 'futurePassengers'] = vehicle_to_passengers.get(v, 0)
    vehicle_to_passengers[v] = int(row['numPassengers'])

def pte_to_icon_type(path_traversal_event):
    now_passengers = path_traversal_event['numPassengers']
    future_passengers = path_traversal_event['futurePassengers']
    if now_passengers == 1.0:
        return "RH_ps1"
    elif now_passengers > 1.0:
        return "RH_ps2"
    elif now_passengers == 0.0 and future_passengers == 1.0:
        return "RH_dh"
    elif now_passengers == 0.0 and future_passengers == 0.0:
        return "RH_rp"
    else:
        print(f"!!!Unexpected values of passengers: now_passengers == {now_passengers} and future_passengers == {future_passengers}")
        return ""

all_rh['Type'] = all_rh.apply(pte_to_icon_type, axis=1)

all_rh.head(2)

,time,type,currentTourMode,vehicleType,links,numPassengers,length,primaryFuel,riders,fromStopIndex,seatingCapacity,tollPaid,secondaryFuelLevel,primaryFuelLevel,endY,endX,startY,startX,capacity,arrivalTime,departureTime,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,toStopIndex,driver,vehicle,mode,futurePassengers,Type
17410,18236.0,PathTraversal,NaN,RH_Car,"78200,90990,20048,70824,12304,50520,43388,43390,43392,20038,12539,3048,51038,1547",0.0,465.537,1.701994e+06,NaN,NaN,4.0,0.0,0.0,3.654278e+09,37.738223,-122.425274,37.737731,-122.424961,4.0,18236.0,18200.0,"11.659,0.668,1.297,1.177,0.664,0.794,0.319,0.133,0.319,1.884,8.888,7.326,6.269,6.448",0.0,None,Gasoline,NaN,rideHailAgent-021800-2013000077130-0-7568754,rideHailVehicle-021800-2013000077130-0-7568754@GlobalRHM,car,1.0,RH_dh
17444,18255.0,PathTraversal,NaN,RH_Car,"7443,7441,53760,53762,53764,86958,70868,86954,26858,70866,35084,86924,35086,35088,35090,35068,35070,35072,86970,35074,35076,19619,19617,19615,19613,19611,19609",0.0,1243.400,4.545846e+06,NaN,NaN,4.0,0.0,0.0,3.651434e+09,37.770170,-122.453756,37.772519,-122.442404,4.0,18255.0,18200.0,"9.498,9.393,4.763,1.182,5.905,0.241,0.409,1.24,1.288,2.822,5.462,0.485,3.868,2.069,1.789,4.173,2.137,1.326,0.729,0.221,1.635,0.318,0.525,0.781,2.017,0.275,0.263",0.0,None,Gasoline,NaN,rideHailAgent-016700-2013001237688-0-7813295,rideHailVehicle-016700-2013001237688-0-7813295@GlobalRHM,car,1.0,RH_dh


In [8]:
rh_icons = [
    {
        "Type":"RH_ps1",
        "BackgroundColor":"c4c4c4",
        "Label":"RH with 1 passenger",
        "Icon":"Triangle"
    },
    {
        "Type":"RH_ps2",
        "BackgroundColor":"888888",
        "Label":"RH with few passengers",
        "Icon":"Triangle"
    },
    {
        "Type":"RH_dh",
        "BackgroundColor":"fccf03",
        "Label":"RH deadheading",
        "Icon":"Triangle"
    },
    {
        "Type":"RH_rp",
        "BackgroundColor":"fc0f03",
        "Label":"RH repositioning",
        "Icon":"Triangle"
    }
]


scenario.add_trajectories_from_pte(all_rh, rh_icons, 'RH')

 -> got 8461 trajectories
 -> adding trajectory with 8461 records, with 4 icons


### BUS PT based on passengers

In [9]:
bus_pte = all_pte[all_pte['mode'] == 'bus'].copy()
bus_pte['numPassengers'].value_counts()

0.0    354187
1.0        53
2.0        40
Name: numPassengers, dtype: int64

In [10]:
bus_pte = all_pte[all_pte['mode'] == 'bus'].copy()

def pte_to_icon_type(path_traversal_event):
    passengers = path_traversal_event['numPassengers']
    if passengers == 0.0:
        return "BUS0"
    elif passengers == 1.0:
        return "BUS1"
    else:
        return "BUS2"

bus_pte['Type'] = bus_pte.apply(pte_to_icon_type, axis=1)
bus_pte['Type'].value_counts()

BUS0    354187
BUS1        53
BUS2        40
Name: Type, dtype: int64

In [11]:
icons = [
    {
        "Type":"BUS0",
        "BackgroundColor":"c4c4c4",
        "Label":"empty bus",
        "Icon":"Triangle"
    },
    {
        "Type":"BUS1",
        "BackgroundColor":"fccf03",
        "Label":"bus with 1 passenger",
        "Icon":"Triangle"
    },
    {
        "Type":"BUS2",
        "BackgroundColor":"fc0f03",
        "Label":"bus with 2 passengers",
        "Icon":"Triangle"
    }
]

scenario.add_trajectories_from_pte(bus_pte, icons, 'BUS')

 -> got 354280 trajectories
 -> adding trajectory with 354280 records, with 3 icons


### the rest of events

In [12]:
is_rh = all_pte['driver'].str.startswith('rideHailAgent')
is_bus = all_pte['mode'] == 'bus'

rest_pte = all_pte[~is_rh & ~is_bus].copy()
display(rest_pte['mode'].value_counts())
rest_pte.head(2)

car     1574
walk     341
Name: mode, dtype: int64

,time,type,currentTourMode,vehicleType,links,numPassengers,length,primaryFuel,riders,fromStopIndex,seatingCapacity,tollPaid,secondaryFuelLevel,primaryFuelLevel,endY,endX,startY,startX,capacity,arrivalTime,departureTime,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,toStopIndex,driver,vehicle,mode
18037,18702.0,PathTraversal,car,Car,"64930,64932,64924,64926,64928,32033,32031,32029,32027,32025,32016,32018,85698,32020,32022,70396,70398,6550,6552,6554,70392,70394,70388,70390,70380,70382,65986,65988,65990,52467,52465,52463,52461,64825,64823,81269,64821,64819,64817,63841,63839,63837,63835,63833,63831,63829,63580,63582,52620,52622,81644,18416,81648,63576,63578,52530,52532,52534,63632,63634,23210,24254,35446,35442,28412,28414,28402,28404,28406,20688,15648,20626,20624,35444,35460,35448,35450,24072,24074,35452,35454,35456,63418,366,70818,32218,42880,12554,65478,90070,90066,41108,41110,41112,72532,83374,83358,72534,83362,83366,20176,89532,20178,39162,90700,39164,90704,39166,39168,39170,90692,39172,90696,2872,2874",0.0,14917.169,5.453687e+07,NaN,NaN,4.0,0.0,0.0,3.601443e+09,37.716109,-122.468023,37.789645,-122.388406,4.0,18702.0,18120.0,"1.099,3.805,3.547,5.101,0.962,0.71,0.24,0.674,0.226,0.609,0.959,1.279,6.076,5.061,1.035,1.025,12.435,0.448,2.366,1.187,2.339,0.857,0.827,14.684,0.327,0.402,1.601,0.74,0.89,0.427,7.582,4.798,2.555,1.834,2.075,0.515,2.211,2.903,5.647,2.686,2.408,0.293,2.902,0.589,0.651,1.726,3.318,1.63,0.832,0.424,0.441,0.616,1.518,1.187,0.388,0.338,1.01,4.807,5.73,8.914,2.847,26.724,2.835,11.491,8.439,5.622,8.91,19.388,5.056,43.627,13.645,10.472,9.717,3.123,4.983,31.406,20.227,1.865,26.283,21.258,30.947,19.972,11.835,5.723,2.513,4.229,1.095,1.68,1.215,1.466,1.715,4.476,8.623,0.161,0.381,9.799,0.413,0.441,11.34,0.64,0.68,3.201,1.092,2.398,0.405,0.609,2.609,3.214,3.209,2.79,0.423,0.422,2.805,0.185,18.112",0.0,None,Gasoline,NaN,061500-2012001149313-0-1191940,369-2,car
18087,18720.0,PathTraversal,car,Car,"2874,2876",0.0,198.613,7.261252e+05,NaN,NaN,4.0,0.0,0.0,3.600717e+09,37.717896,-122.468042,37.716109,-122.468023,4.0,18720.0,18702.0,"18.112,17.771",0.0,None,Gasoline,NaN,061500-2012001149313-0-1191940,369-2,car


In [13]:
rest_pte['Type'] = rest_pte.apply(lambda e: 'car' if e['mode']=='car' else 'walk', axis=1)
rest_pte['Type'].value_counts()

car     1574
walk     341
Name: Type, dtype: int64

In [14]:
icons = [
    {
        "Type":"car",
        "BackgroundColor":"333333",
        "Label":"regular car",
        "Icon":"Triangle"
    },
    {
        "Type":"walk",
        "BackgroundColor":"339933",
        "Label":"regular pedastrian",
        "Icon":"Triangle"
    }
]

scenario.add_trajectories_from_pte(rest_pte, icons, 'walk_car')

 -> got 1915 trajectories
 -> adding trajectory with 1915 records, with 2 icons


### finish

In [15]:
scenario.write_scenario()
scenario.pack_to_archive()

 -> 1915 trajectories written to /home/jovyan/visualisation/sflight-1k_rh_bus_other/trajectory_walk_car_5.csv ...
 -> trajectories settings written to /home/jovyan/visualisation/sflight-1k_rh_bus_other/trajectory_walk_car_5_settings.json
 -> 354280 trajectories written to /home/jovyan/visualisation/sflight-1k_rh_bus_other/trajectory_BUS_4.csv ...
 -> trajectories settings written to /home/jovyan/visualisation/sflight-1k_rh_bus_other/trajectory_BUS_4_settings.json
 -> 8461 trajectories written to /home/jovyan/visualisation/sflight-1k_rh_bus_other/trajectory_RH_3.csv ...
 -> trajectories settings written to /home/jovyan/visualisation/sflight-1k_rh_bus_other/trajectory_RH_3_settings.json
 -> 0 events written to /home/jovyan/visualisation/sflight-1k_rh_bus_other/events_empty_1.csv ...
 -> events settings written to /home/jovyan/visualisation/sflight-1k_rh_bus_other/events_empty_1_settings.json
 -> scenario files written to /home/jovyan/visualisation/sflight-1k_rh_bus_other
 -> scenario pac

## all PT events

In [13]:
beam_output = "../beam_root/output/sf-light/sf-light-1k-xml__2024-05-06_18-09-08_xjf"
output_folder_path = "sflight-1k_bus_car_walk_all_pte_by_mode"
# beam_output = "../downloaded_data/sfbay/sfbay-freight-23Jan24-Base__2024-01-31_18-10-36_gfh"
# output_folder_path = "sfbay-freight-23Jan24-Base_bus_car_walk_all_pte_by_mode_025_sample"

beam_crs = 26910

scenario = Scenario(beam_output, beam_crs, output_folder_path)
scenario.read_beam_output()
scenario.write_network()

 -> 'image_folders/EventIcons' copied to '/home/jovyan/visualisation/sflight-1k_bus_car_walk_all_pte_by_mode/EventIcons' successfully.
 -> 'image_folders/TrajectoryIcons' copied to '/home/jovyan/visualisation/sflight-1k_bus_car_walk_all_pte_by_mode/TrajectoryIcons' successfully.
 -> the output path set to '/home/jovyan/visualisation/sflight-1k_bus_car_walk_all_pte_by_mode'
 -> reading beam output folder
 -> read network (94350) from '../beam_root/output/sf-light/sf-light-1k-xml__2024-05-06_18-09-08_xjf/network.csv.gz'
 -> network written to '/home/jovyan/visualisation/sflight-1k_bus_car_walk_all_pte_by_mode/network.csv'
 -> network settings written to /home/jovyan/visualisation/sflight-1k_bus_car_walk_all_pte_by_mode/network_settings.json
 -> dynamic network written to /home/jovyan/visualisation/sflight-1k_bus_car_walk_all_pte_by_mode/dynamic_network.csv
 -> dynamic network settings written to /home/jovyan/visualisation/sflight-1k_bus_car_walk_all_pte_by_mode/dynamic_network_settings.j

In [14]:
all_pte = read_pte_events(beam_output, 0)

print(f" ->> total number of rows in DF {len(all_pte)}")
display(all_pte.head(2))

all_pte['mode'].value_counts()

 ->> total number of rows in DF 364664


,startX,startY,endX,endY,time,type,driver,vehicle,currentTourMode,vehicleType,links,numPassengers,length,primaryFuel,riders,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,secondaryFuelLevel,primaryFuelLevel,capacity,arrivalTime,departureTime,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,mode
16036,-122.418106,37.748574,-122.420890,37.744058,14853.0,PathTraversal,TransitDriverAgent-SF:7596499,SF:7596499,NaN,BUS-DEFAULT,"51087,50223,51085,94245,51083,51081,47225,88927,47223",0.0,518.513,1.039515e+07,NaN,1.0,0.0,19.0,0.0,0.0,2.998960e+10,29.0,14853.0,14760.0,"5.875,5.875,5.875,5.875,5.875,5.875,5.875,5.875,0",0.0,None,Diesel,bus
16037,-122.420258,37.745036,-122.421952,37.742415,14923.0,PathTraversal,TransitDriverAgent-SF:7596499,SF:7596499,NaN,BUS-DEFAULT,"47223,47221,47219,47217,88759",0.0,205.071,4.111263e+06,NaN,2.0,1.0,19.0,0.0,0.0,2.998549e+10,29.0,14923.0,14853.0,"8.5,8.5,8.5,8.5,0",0.0,None,Diesel,bus


bus     354280
car      10041
walk       343
Name: mode, dtype: int64

In [5]:
import math

all_vehicle_ids = all_pte['vehicle'].unique()
random.shuffle(all_vehicle_ids)

number_of_samples = math.floor(len(all_vehicle_ids) * 0.25)
selected_vehicle_ids = set(all_vehicle_ids[:number_of_samples])

print(f"selected:{len(selected_vehicle_ids)}, total:{len(all_vehicle_ids)}, ratio:{len(selected_vehicle_ids) / len(all_vehicle_ids)}, sanity:{(len(selected_vehicle_ids) - number_of_samples) == 0}")

selected:248941, total:995766, ratio:0.2499994978739985, sanity:True


In [16]:
all_pte.groupby('mode')['numPassengers'].value_counts()

mode  numPassengers
bus   0.0              354183
      1.0                  60
      2.0                  37
car   0.0                5907
      1.0                4134
walk  1.0                 343
Name: numPassengers, dtype: int64

In [6]:
# is_selected = all_pte['vehicle'].isin(selected_vehicle_ids)
# sampled_pte = all_pte[is_selected]

icons = [
    {
        "Type":"BUS",
        "BackgroundColor":"c4c4c4",
        "Label":"any bus",
        "Icon":"Triangle"
    },
    {
        "Type":"CAR",
        "BackgroundColor":"fccf03",
        "Label":"any car",
        "Icon":"Triangle"
    },
    {
        "Type":"WALK",
        "BackgroundColor":"fc0f03",
        "Label":"walk",
        "Icon":"Triangle"
    }
]

def pte_to_icon_type(path_traversal_event):
    mode = path_traversal_event['mode']
    if mode == 'bus':
        return "BUS"
    elif mode == 'car' or mode == 'car_hov2' or mode == 'car_hov3':
        return "CAR"
    else:
        return "WALK"

def pte_to_progress_bar(pte):
    return "None"


scenario.set_trajectoris(sampled_pte, pte_to_icon_type, pte_to_progress_bar, icons)
scenario.write_scenario()
scenario.pack_to_archive()

 -> got 1721520 trajectories
 -> network already written out to file.
 -> dynamic network already written out to file.
 -> 1721520 trajectories written to /home/jovyan/visualisation/sfbay-freight-23Jan24-Base_bus_car_walk_all_pte_by_mode_025_sample/trajectories.csv ...
 -> trajectories settings written to /home/jovyan/visualisation/sfbay-freight-23Jan24-Base_bus_car_walk_all_pte_by_mode_025_sample/trajectories_settings.json
 -> 0 events written to /home/jovyan/visualisation/sfbay-freight-23Jan24-Base_bus_car_walk_all_pte_by_mode_025_sample/events.csv ...
 -> events settings written to /home/jovyan/visualisation/sfbay-freight-23Jan24-Base_bus_car_walk_all_pte_by_mode_025_sample/events_settings.json
 -> scenario files written to /home/jovyan/visualisation/sfbay-freight-23Jan24-Base_bus_car_walk_all_pte_by_mode_025_sample
 -> scenario packed to '/home/jovyan/visualisation/sfbay-freight-23Jan24-Base_bus_car_walk_all_pte_by_mode_025_sample.zip'


In [7]:
! ls -lahS | head -5

total 2.7G
-rw-r--r--  1 jovyan users 1.4G Jun 26 14:08 sfbay-freight-23Jan24-Base_bus_car_walk_all_pte_by_mode.zip
-rw-r--r--  1 jovyan users 717M Jun 27 16:50 sfbay-freight-23Jan24-Base_bus_car_walk_all_pte_by_mode_05_sample.zip
-rw-r--r--  1 jovyan users 361M Jun 27 17:48 sfbay-freight-23Jan24-Base_bus_car_walk_all_pte_by_mode_025_sample.zip
-rw-r--r--  1 jovyan users 147M Jun 27 15:42 sfbay-freight-23Jan24-Base_bus_car_walk_all_pte_by_mode_01_sample.zip


## all bus\rh PT events with passengers + all car PT events 

In [ ]:
beam_output = "../beam_root/output/sf-light/sf-light-1k-xml__2024-05-06_18-09-08_xjf"
output_folder_path = "sflight-1k_with_passengers_only_bus_car_walk_pte_by_mode"
beam_crs = 26910

scenario = Scenario(beam_output, beam_crs, output_folder_path)
scenario.read_beam_output()

all_pte = read_pte_events(beam_output, 0)

single_car_types = set(['BEV','Car','PHEV'])
single_car = all_pte['vehicleType'].isin(single_car_types)
has_passengers = all_pte['numPassengers'] > 0.0
pte = all_pte[single_car | has_passengers].copy()

icons = [
    {
        "Type":"BUS",
        "BackgroundColor":"c4c4c4",
        "Label":"any bus with passengers",
        "Icon":"Triangle"
    },
    {
        "Type":"CAR",
        "BackgroundColor":"fccf03",
        "Label":"RH with passengers or any car",
        "Icon":"Triangle"
    },
    {
        "Type":"WALK",
        "BackgroundColor":"fc0f03",
        "Label":"walk",
        "Icon":"Triangle"
    }
]

def pte_to_icon_type(path_traversal_event):
    mode = path_traversal_event['mode']
    if mode == 'bus':
        return "BUS"
    elif mode == 'car':
        return "CAR"
    else:
        return "WALK"

def pte_to_progress_bar(pte):
    return "None"


scenario.set_trajectoris(pte, pte_to_icon_type, pte_to_progress_bar, icons)
scenario.write_scenario()
scenario.pack_to_archive()

## selected actor trip

In [ ]:
#
# HOW TO USE read_events_enhanced_events_trips function
#
## 0. execute this cell
## 1. execute code in cell 1 with correct path to events 
## 2. execute code in cell 2
## 3. execute code in cell 3 (optionally change selected_trip_id in cell 3)
#
## results of cell 1 include: whole original events DF, enhanced events DF and all trips DF
## enhanced events has trip_id for some events types, improved order and one PathTraversal event per rider (instead of one PathTraversal event per vehicle)
## %%time - is a magic command which calculates how long the execution of the whole cell took, this command should be the first row in a cell

beam_output = "../beam_root/output/sf-light/sf-light-1k-xml__2024-05-06_18-09-08_xjf"
output_folder_path = "sflight-1k_selected_actor_trip_only"
beam_crs = 26910

scenario = Scenario(beam_output, beam_crs, output_folder_path)
scenario.read_beam_output()
print('scenario prepared')

In [ ]:
%%time
## CELL 1 - read all events
path1 = f"{beam_output}/ITERS/it.0/0.events.csv.gz"
events_original, events_enhanced, all_trips = read_events_enhanced_events_trips(path1)

## adding len of the trip
all_trips['trip_len'] = all_trips.apply(lambda r: len(r['index']), axis=1)
all_trips['veh_number'] = all_trips.apply(lambda r: len(r['trip_vehicles']), axis=1)
    

print(f"Size of original events DF: {len(events_original)}, enhanced events DF: {len(events_enhanced)}, all trips DF: {len(all_trips)}")
# display(events_original.head(2))
# display(events_enhanced.head(2))
display(all_trips.head(2))

In [ ]:
%%time
### CELL 2 - find the required trip to use
more_vehicles = max(all_trips['veh_number'].unique())
selected_trips = all_trips[all_trips['veh_number'] == more_vehicles - 1]
print(f"there are {len(selected_trips)} selected trips")

display(selected_trips.head(2))

In [ ]:
%%time
### CELL 3 - show selected trip_id
selected_trip_id = random.choice(selected_trips['trip_id'].unique())
display(f"Trip Id {selected_trip_id}")
trip_df = get_trip_df(events_enhanced, selected_trip_id)
display(f"Number of events in it: {len(trip_df)}")
columns = ['person', 'trip_id', 'type', 'vehicle', 'mode', 'time', 'departureTime', 'arrivalTime', 'length', 'links', 'linkTravelTime']
display(trip_df[columns])

In [ ]:
selected_person = "032802-2015000455334-0-7952563"

is_pte = events_original['type'] == 'PathTraversal'
with_links = events_original['links'].notna()
all_pte = events_original[is_pte & with_links].dropna(axis=1, how='all').copy()

def is_selected(row):
    if row['driver'] == selected_person:
        return True
    riders = row['riders']
    if riders and selected_person in str(riders):
        return True
    return False

all_pte['selected'] = all_pte.apply(is_selected, axis=1)

time_min = all_pte[all_pte['selected'] == True]['time'].min()
time_max = all_pte[all_pte['selected'] == True]['time'].max()

time_more_than_min = all_pte['time'] > (time_min - 10 * 60.0)
time_less_than_max = all_pte['time'] < (time_max + 10 * 60.0)

all_pte = all_pte[time_more_than_min & time_less_than_max]

print(f"there are {len(all_pte)} events in dataframe")
display(all_pte.head(2))
display(all_pte['selected'].value_counts())

In [ ]:
icons = [
    {
        "Type":"WALK",
        "BackgroundColor":"c4c4c4",
        "Label":"walk of selected agent",
        "Icon":"Triangle"
    },
    {
        "Type":"RH",
        "BackgroundColor":"bf2e2e",
        "Label":"RH of selected agent",
        "Icon":"Triangle"
    },
    {
        "Type":"REST",
        "BackgroundColor":"919191",
        "Label":"the rest of PT event",
        "Icon":"Triangle"
    }
]

def pte_to_icon_type(path_traversal_event):
    if path_traversal_event['selected'] == False:
        return "REST"
    
    mode = path_traversal_event['mode']
    if mode == 'walk':
        return "WALK"
    elif mode == 'car':
        return "RH"
    else:
        return "REST"

def pte_to_progress_bar(pte):
    return "None"

selected_pte_only = all_pte[all_pte['selected'] == True].copy()

scenario.set_trajectoris(selected_pte_only, pte_to_icon_type, pte_to_progress_bar, icons)
scenario.write_scenario()
scenario.pack_to_archive()

In [ ]:
selected_pte_only = all_pte[all_pte['selected'] == True].copy()
selected_pte_only

In [ ]:
events_original[events_original['driver'] == "032802-2015000455334-0-7952563"].dropna(axis=1, how='all')

## subway as events

In [6]:
beam_output = "../beam_root/output/sf-light/sf-light-1k-walk-transit__2024-07-01_19-15-50_dtv"
output_folder_path = "sflight-1k_subway_plus_walk"
# beam_output = "../downloaded_data/sfbay/sfbay-freight-23Jan24-Base__2024-01-31_18-10-36_gfh"
# output_folder_path = "sfbay-freight-23Jan24-Base_bus_car_walk_all_pte_by_mode_025_sample"

beam_crs = 26910

scenario = Scenario(beam_output, beam_crs, output_folder_path)
scenario.read_beam_output()
scenario.write_network()

 -> the output path set to '/home/jovyan/visualisation/sflight-1k_subway_plus_walk'
 -> reading beam output folder
 -> read network (94350) from '../beam_root/output/sf-light/sf-light-1k-walk-transit__2024-07-01_19-15-50_dtv/network.csv.gz'
 -> network written to '/home/jovyan/visualisation/sflight-1k_subway_plus_walk/network.csv'
 -> network settings written to /home/jovyan/visualisation/sflight-1k_subway_plus_walk/network_settings.json
 -> dynamic network written to /home/jovyan/visualisation/sflight-1k_subway_plus_walk/dynamic_network.csv
 -> dynamic network settings written to /home/jovyan/visualisation/sflight-1k_subway_plus_walk/dynamic_network_settings.json


In [8]:
events_path = get_events_file_path(beam_output, 0)
all_events = read_events(events_path)
all_events.head()

,person,vehicle,time,type,vehicleType,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,actType,mode,currentTourMode,availableAlternatives,location,personalVehicleAvailable,length,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,links,numPassengers,primaryFuel,riders,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,secondaryFuelLevel,primaryFuelLevel,endY,endX,startY,startX,capacity,arrivalTime,departureTime,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,driver,link,reason,cost,departTime,requireWheelchair
0,NaN,6-0,0.0,ParkingEvent,NaN,100896,None,FlatFee,Residential,37.739653,-122.427774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,0.0,NaN,NaN
1,NaN,6-1,0.0,ParkingEvent,NaN,100896,None,FlatFee,Residential,37.739653,-122.427774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,0.0,NaN,NaN
2,NaN,5-0,0.0,ParkingEvent,NaN,100571,None,FlatFee,Residential,37.790406,-122.421767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,0.0,NaN,NaN
3,NaN,5-1,0.0,ParkingEvent,NaN,100571,None,FlatFee,Residential,37.790406,-122.421767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,0.0,NaN,NaN
4,NaN,1-0,0.0,ParkingEvent,NaN,100599,None,FlatFee,Residential,37.784072,-122.420995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,0.0,NaN,NaN
